In [1]:
import seq2seq
import dataset
import pickle
import torch

/usr/lib/python3/dist-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
# load the data for building vocabulary
data_dir = '/home/lyu/OCR-post/OCR_posthoc/data/all_books.PKL'
model_dir = '/home/lyu/OCR-post/OCR_posthoc/Model/all_books0.model'
with open(data_dir, 'rb')as f:
    Train, _ = pickle.load(f)  # Train, Test
Train_data = Train[0] # one group with 12 books
tokenizer = 'char' # we use character tokenization, thus the sequence length is in character level instead of word, vocabulary is also characters.
features = 100000  # maximum vocabulary size
DATA = dataset.LoadData(tokenizer, features)
DATA.build_vocab_on_the_fly(Train_data) # initialize vocabs


# initialize hyperparameters for model
batch = 64
inp_dim = out_dim = len(DATA.vocab)
embed = 128
enc = dec = 256
dropout = 0.3
epoch = 10
clip = 0.1
sparse = 1
tf = 0.5  # teacher forcing ratio, 0.5 by default, only used for training
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# initialize and load model
OCR = seq2seq.Train(inp_dim, out_dim, embed, enc, dec, dropout, dropout, epoch, clip, sparse, tf, DATA, batch, device, model_dir)

vocabulary size:  141 | max setence length: 98
The model has 1015053 trainable parameters
loading model from  /home/lyu/OCR-post/OCR_posthoc/Model/all_books0.model


In [3]:
# apply the model for sentence correction. here we can do it in batch level instead of sentence by sentence.
sentences = [u'eine Thatimputiren könnea eben daher', u'Ich habeeinHund', u'denen von unserkauffeten eben']  # a list of sentences. 
OCR.translate_in_batch(sentences)


tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.5354, -1.6452, -0.6613,  ..., -2.7738, -2.0779, -2.4496],
         [-1.2474, -1.1460, -0.5197,  ..., -2.2670, -1.5053, -1.4116],
         [-2.1479, -1.6316, -0.2487,  ..., -2.9168, -2.6217, -2.6713]],

        [[-0.9463, -1.6029, -0.5093,  ..., -2.8489, -2.1247, -1.7661],
         [-1.8023, -1.5260, -0.1650,  ..., -2.8891, -2.2788, -2.0913],
         [-1.4844, -1.7535, -0.2369,  ..., -2.8000, -2.2330, -2.8282]],

        ...,

        [[-1.3036, -1.5950,  0.7399,  ..., -2.9613, -2.1947, -2.6382],
         [-1.0119, -1.5867,  0.3935,  ..., -2.3107, -1.7496, -2.3703],
         [-0.8339, -1.4646,  0.3158,  ..., -1.9115, -2.1101, -1.5750]],

        [[-1.2221, -1.4767,  0.6938,  ..., -2.8638, -2.0826, -2.4459],
         [-1.0033, -1.6616,  0.3390,  ..., -2.2174, -1.

['eine Thatimputiren koͤnne aben daher',
 'Jch habe ein Hund',
 'denen von uns erkauffeten eben']

In [2]:
import pickle

In [4]:
data_dir_2 = '/home/lyu/OCR-post/OCR_posthoc/data/Experiment_corpus.PKL'
with open(data_dir_2,'rb')as f:
    Train_groups, Test_groups = pickle.load(f)

In [19]:
test = Test_groups[0][:65]

In [20]:
test_inp = [a[0] for a in test]
test_out = [a[1] for a in test]

In [21]:
OCR.translate_in_batch(test_inp)

Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f3b0960ac80>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


RuntimeError: CUDA out of memory. Tried to allocate 15.75 MiB (GPU 0; 1.95 GiB total capacity; 1.26 GiB already allocated; 3.31 MiB free; 10.11 MiB cached)